In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\alext\Desktop\quantum_research\data\cleaned_patents(not_labeled).csv",
                   on_bad_lines='skip')

In [3]:
data.head()

,#,Publication Year,Title,Abstract,Applicants,Inventors,Country
0,1,2023,"QUANTUM COMPUTING PROGRAM, QUANTUM COMPUTING A...",to obtain a solution even when the coefficient...,['NAT UNIV YOKOHAMA'],"['ISHIKAWA NAOKI', 'NORIMOTO MASAYA']",Japan
1,2,2023,"QUANTUM CIRCUIT, QUANTUM COMPUTING ELEMENT, QU...",provided are a quantum circuit a quantum compu...,['AIST'],['SAIDA DAISUKE'],Wipo (International - Pct)
2,3,2024,"QUANTUM CIRCUIT, QUANTUM COMPUTING ELEMENT, QU...",provided is a quantum circuit a quantum comput...,['AIST'],['SAIDA DAISUKE'],United States
3,4,2023,"QUANTUM COMPUTING DEVICE, QUANTUM COMPUTING SY...",this quantum computing device in a quantum com...,['NIPPON TELEGRAPH & TELEPHONE'],"['MISHCHENKO PETR', 'KUSAGAWA KEITA']",Wipo (International - Pct)
4,5,2024,"QUANTUM COMPUTING SYSTEM, QUANTUM COMPUTING DE...",a quantum computing system according to one as...,['NIPPON TELEGRAPH & TELEPHONE'],"['YAMAMOTO KAORU', 'ENDO SUGURU', 'SUZUKI YASU...",Wipo (International - Pct)


In [6]:

labels = [
    "quantum optimization",
    "quantum algorithms",
    "quantum computing theory",
    "quantum resource theory",
    "quantum coherence",
    "quantum control",
    "quantum security tools",
    "quantum communication",
    "quantum cryptography",
    "quantum sensing",
    "quantum compilation",
    "quantum data structures",
    "quantum signal processing",
    "quantum distributed computing",
    "quantum simulation",
    "quantum error correction",
    "quantum software tools",
    "quantum computational complexity",
    "quantum error mitigation",
    "quantum state tomography",
    "quantum computing",
    "quantum surface codes",
    "quantum computing applications",
    "quantum hardware",
    "quantum thermodynamics",
    "quantum computing foundations",
    "quantum image processing",
    "quantum information theory",
    "quantum visualization tools",
    "quantum computing hardware (hybrid)",
    "quantum machine learning",
    "quantum workforce development",
    "quantum computing in ai",
    "quantum metrology",
    "quantum noise spectroscopy",
    "quantum conversion",
    "quantum logic gates",
    "quantum reinforcement learning",
    "quantum fidelity estimation"
]


In [7]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time

In [8]:
API_KEY = "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"
MODEL = "anthropic/claude-3-haiku"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "HTTP-Referer": "https://openrouter.ai",
    "Content-Type": "application/json"
}

In [9]:
prompt_template = """
You are an expert in quantum computing.
Classify the following paper by choosing **exactly one** label from the list below.
Only output the label itself—no punctuation, quotes, or extra text.

Available labels:
{}

Title: {}
Abstract: {}
"""

In [13]:
import requests, json, time
predicted_labels = []
checkpoint_interval = 4000

In [14]:
for i, row in tqdm(data.iterrows(), total=len(data), desc="Classifying"):
    title = str(row["Title"])
    abstract = str(row["Abstract"])

    prompt = prompt_template.format("\n".join(labels), title.strip(), abstract.strip())

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            data=json.dumps(payload),
            timeout=30  
        )
        result = response.json()
        label = result["choices"][0]["message"]["content"].strip()

        # Optional: ensure the label is one of the allowed ones
        if label not in labels:
            print(f"Warning: Unexpected label at index {i}: '{label}'")
            label = "invalid_label"

    except Exception as e:
        print(f"Error at index {i}: {e}")
        label = "error"

    predicted_labels.append(label)

    # Save checkpoint
    if (i + 1) % checkpoint_interval == 0:
        temp = data.iloc[:i + 1].copy()
        temp["Label"] = predicted_labels
        temp.to_csv(f"checkpoint_up_to_{i + 1}.csv", index=False)
        print(f"→ Saved checkpoint for rows 0–{i + 1}")

    time.sleep(0.2)

# Final save
data["Label"] = predicted_labels
data.to_csv("abstracts_with_labels_full.csv", index=False)
print("✅ Completed classification of all rows and saved final CSV.")

Classifying:   0%|                                                               | 18/47871 [00:23<16:40:46,  1.25s/it]

Classifying:   0%|                                                               | 56/47871 [01:26<19:13:02,  1.45s/it]

Classifying:   0%|                                                               | 89/47871 [02:16<19:15:43,  1.45s/it]

Classifying:   1%|▎                                                             | 258/47871 [07:19<14:40:20,  1.11s/it]

Classifying:   1%|▎                                                             | 265/47871 [07:35<22:03:57,  1.67s/it]

...

Classifying: 100%|███████████████████████████████████████████████████████████▉| 47778/47871 [16:04:13<01:24,  1.10it/s]

Classifying: 100%|███████████████████████████████████████████████████████████▉| 47864/47871 [16:05:38<00:07,  1.09s/it]

Classifying: 100%|███████████████████████████████████████████████████████████▉| 47868/47871 [16:05:42<00:02,  1.04it/s]

Classifying: 100%|████████████████████████████████████████████████████████████| 47871/47871 [16:05:45<00:00,  1.21s/it]


✅ Completed classification of all rows and saved final CSV.


In [15]:
df = pd.read_csv(r"C:\Users\alext\Desktop\quantum_research\abstracts_with_labels_full.csv",
                   on_bad_lines='skip')

In [16]:
df.head()

,#,Publication Year,Title,Abstract,Applicants,Inventors,Country,Label
0,1,2023,"QUANTUM COMPUTING PROGRAM, QUANTUM COMPUTING A...",to obtain a solution even when the coefficient...,['NAT UNIV YOKOHAMA'],"['ISHIKAWA NAOKI', 'NORIMOTO MASAYA']",Japan,quantum optimization
1,2,2023,"QUANTUM CIRCUIT, QUANTUM COMPUTING ELEMENT, QU...",provided are a quantum circuit a quantum compu...,['AIST'],['SAIDA DAISUKE'],Wipo (International - Pct),quantum computing
2,3,2024,"QUANTUM CIRCUIT, QUANTUM COMPUTING ELEMENT, QU...",provided is a quantum circuit a quantum comput...,['AIST'],['SAIDA DAISUKE'],United States,quantum computing
3,4,2023,"QUANTUM COMPUTING DEVICE, QUANTUM COMPUTING SY...",this quantum computing device in a quantum com...,['NIPPON TELEGRAPH & TELEPHONE'],"['MISHCHENKO PETR', 'KUSAGAWA KEITA']",Wipo (International - Pct),quantum error correction
4,5,2024,"QUANTUM COMPUTING SYSTEM, QUANTUM COMPUTING DE...",a quantum computing system according to one as...,['NIPPON TELEGRAPH & TELEPHONE'],"['YAMAMOTO KAORU', 'ENDO SUGURU', 'SUZUKI YASU...",Wipo (International - Pct),quantum computing


In [21]:
df["Title"] = df["Title"].str.lower()

In [22]:
import ast

df["Applicants"] = data["Applicants"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["Inventors"] = data["Inventors"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [23]:
def clean_names(name_list):
    return [name.strip().title() for name in name_list if isinstance(name, str)]

df["Applicants"] = df["Applicants"].apply(clean_names)
df["Inventors"] = df["Inventors"].apply(clean_names)


In [24]:
df.head()

,#,Publication Year,Title,Abstract,Applicants,Inventors,Country,Label
0,1,2023,"quantum computing program, quantum computing a...",to obtain a solution even when the coefficient...,[Nat Univ Yokohama],"[Ishikawa Naoki, Norimoto Masaya]",Japan,quantum optimization
1,2,2023,"quantum circuit, quantum computing element, qu...",provided are a quantum circuit a quantum compu...,[Aist],[Saida Daisuke],Wipo (International - Pct),quantum computing
2,3,2024,"quantum circuit, quantum computing element, qu...",provided is a quantum circuit a quantum comput...,[Aist],[Saida Daisuke],United States,quantum computing
3,4,2023,"quantum computing device, quantum computing sy...",this quantum computing device in a quantum com...,[Nippon Telegraph & Telephone],"[Mishchenko Petr, Kusagawa Keita]",Wipo (International - Pct),quantum error correction
4,5,2024,"quantum computing system, quantum computing de...",a quantum computing system according to one as...,[Nippon Telegraph & Telephone],"[Yamamoto Kaoru, Endo Suguru, Suzuki Yasunari,...",Wipo (International - Pct),quantum computing


In [26]:
df = df[~df["Label"].isin(["invalid_label", "error"])]

In [27]:
print(df["Label"].value_counts())

Label
quantum computing                      6216
quantum computing applications         5256
quantum hardware                       4636
quantum optimization                   4495
quantum sensing                        3685
quantum computing hardware (hybrid)    3531
quantum cryptography                   2948
quantum security tools                 2721
quantum control                        2135
quantum machine learning               1399
quantum communication                  1329
quantum algorithms                     1056
quantum image processing                968
quantum error correction                810
quantum signal processing               797
quantum distributed computing           648
quantum data structures                 523
quantum conversion                      475
quantum simulation                      415
quantum error mitigation                354
quantum state tomography                352
quantum compilation                     309
quantum coherence         

In [28]:
data.to_csv("patents_labeled.csv", index=False)